In [1]:
import pandas as pd
from get_db_url import get_db_url

# Discrete Anomaly Detection Exercises

Use basic probability to identify anomalous requests. Using the methods covered in this lesson, examine the rest of the features in the api access logs data set.

## Acquire the Data

In [2]:
# Acquire the data from the MySQL database
df = pd.read_sql('SELECT * FROM api_access', get_db_url('logs'))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13974 entries, 0 to 13973
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   entry   13974 non-null  object
dtypes: object(1)
memory usage: 109.3+ KB


## Prepare the Data

In [3]:
# function to deal with parsing one entry in our log data
def parse_log_entry(entry):
    parts = entry.split()
    output = {}
    output['ip'] = parts[0]
    output['timestamp'] = parts[3][1:].replace(':', ' ', 1)
    output['request_method'] = parts[5][1:]
    output['request_path'] = parts[6]
    output['http_version'] = parts[7][:-1]
    output['status_code'] = parts[8]
    output['size'] = int(parts[9])
    output['user_agent'] = ' '.join(parts[11:]).replace('"', '')
    return pd.Series(output)

In [4]:
df = df.entry.apply(parse_log_entry)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13974 entries, 0 to 13973
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ip              13974 non-null  object
 1   timestamp       13974 non-null  object
 2   request_method  13974 non-null  object
 3   request_path    13974 non-null  object
 4   http_version    13974 non-null  object
 5   status_code     13974 non-null  object
 6   size            13974 non-null  int64 
 7   user_agent      13974 non-null  object
dtypes: int64(1), object(7)
memory usage: 873.5+ KB


In [5]:
df.head()

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
0,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0
1,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0
2,97.105.19.58,16/Apr/2019 19:34:44,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0
3,97.105.19.58,16/Apr/2019 19:34:46,GET,/api/v1/sales?page=83,HTTP/1.1,200,510003,python-requests/2.21.0
4,97.105.19.58,16/Apr/2019 19:34:48,GET,/api/v1/sales?page=84,HTTP/1.1,200,511963,python-requests/2.21.0


In [6]:
# Let's do some feature engineering and set timestamp as the index
df['size_mb'] = df['size'] / (1024 ** 2)
df.timestamp = pd.to_datetime(df.timestamp)
df = df.set_index('timestamp')

In [7]:
df.head()

,ip,request_method,request_path,http_version,status_code,size,user_agent,size_mb
timestamp,,,,,,,,
2019-04-16 19:34:42,97.105.19.58,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0,0.488753
2019-04-16 19:34:42,97.105.19.58,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0,0.003396
2019-04-16 19:34:44,97.105.19.58,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0,0.486472
2019-04-16 19:34:46,97.105.19.58,GET,/api/v1/sales?page=83,HTTP/1.1,200,510003,python-requests/2.21.0,0.486377
2019-04-16 19:34:48,97.105.19.58,GET,/api/v1/sales?page=84,HTTP/1.1,200,511963,python-requests/2.21.0,0.488246


## Explore request_method

In [8]:
# Let's see the unique values of request_method
df.request_method.value_counts()

GET    13974
Name: request_method, dtype: int64

Nothing to see here

## Explore timestamp

In [9]:
# Let's see if there are any anomalous time stamps. We'll create columns for day of the week and hour first.
df['day_of_week'] = df.index.day_name()
df['hour'] = df.index.hour

In [10]:
df.head()

,ip,request_method,request_path,http_version,status_code,size,user_agent,size_mb,day_of_week,hour
timestamp,,,,,,,,,,
2019-04-16 19:34:42,97.105.19.58,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0,0.488753,Tuesday,19
2019-04-16 19:34:42,97.105.19.58,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0,0.003396,Tuesday,19
2019-04-16 19:34:44,97.105.19.58,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0,0.486472,Tuesday,19
2019-04-16 19:34:46,97.105.19.58,GET,/api/v1/sales?page=83,HTTP/1.1,200,510003,python-requests/2.21.0,0.486377,Tuesday,19
2019-04-16 19:34:48,97.105.19.58,GET,/api/v1/sales?page=84,HTTP/1.1,200,511963,python-requests/2.21.0,0.488246,Tuesday,19


In [11]:
# Let's see the value counts of the day of the week.
df.day_of_week.value_counts()

Tuesday      11947
Wednesday     2027
Name: day_of_week, dtype: int64

Nothing unusual there.

In [12]:
# Let's see the value counts of the hour.
df.hour.value_counts()

20    4587
21    3430
19    2244
22    1684
3      807
1      380
2      284
5      253
4      209
12      70
6       15
0        8
23       2
10       1
Name: hour, dtype: int64

It looks like most requests occur in the hour range 19 through 22. My guess is that this is UTC time. Let's convert to CST.

In [13]:
df.index = df.index - pd.Timedelta('6h')

In [14]:
# Let's now re-do the day_of_week and hour columns
df['day_of_week'] = df.index.day_name()
df['hour'] = df.index.hour

In [15]:
# Now let's see the value counts for day_of_week again.
df.day_of_week.value_counts()

Tuesday      13888
Wednesday       86
Name: day_of_week, dtype: int64

Still nothing particularly unusual except that only a few requests were made on Wednesday.

In [16]:
# Let's see hour value counts again.
df.hour.value_counts().sort_index()

0       15
4        1
6       70
13    2244
14    4587
15    3430
16    1684
17       2
18       8
19     380
20     284
21     807
22     209
23     253
Name: hour, dtype: int64

In [17]:
# Let's try grouping day_of_week and hour.
df.groupby(['day_of_week', 'hour']).size()

day_of_week  hour
Tuesday      13      2244
             14      4587
             15      3430
             16      1684
             17         2
             18         8
             19       380
             20       284
             21       807
             22       209
             23       253
Wednesday    0         15
             4          1
             6         70
dtype: int64

Now we see some meaningful anomalies. Looks like most requests were made between 1:00 PM and 4:59 PM which makes sense since class ends at 5. A few requests were made after 5 (someone didn't want to stop yet). Then, around 7:00 PM until 11 probably a handful of people started working again after dinner. Someone was working on the exercises at 4 in the morning.

In [18]:
# Let's see where that early morning request came from.
df[df.hour == 4]

,ip,request_method,request_path,http_version,status_code,size,user_agent,size_mb,day_of_week,hour
timestamp,,,,,,,,,,
2019-04-17 04:25:50,45.23.250.16,GET,/api/v1/stores,HTTP/1.1,200,1328,python-requests/2.21.0,0.001266,Wednesday,4


## Explore user_agent

In [19]:
# Let's see the unique values in user_agent
df.user_agent.value_counts()

python-requests/2.21.0                                                                                                       12001
python-requests/2.20.1                                                                                                        1911
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36       34
Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0                                               8
Slackbot-LinkExpanding 1.0 (+https://api.slack.com/robots)                                                                       7
Slackbot 1.0 (+https://api.slack.com/robots)                                                                                     6
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36        4
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gec

There are a handful of anomalies here. It looks like most of the anomalies came from web browsers, a few from slackbot, and one person tried using urllib instead of requests.

## A Function for Finding Anomalies

In [32]:
# Let's create a function that will return all anomalous data for a given column provided some threshold for 
# determining what is anomalous.

def get_anomalies(df, groupby_columns, threshold):
    '''
        Group the dataframe by the given columns and return all observations 
        where the proportion of values compared to the whole is less than the 
        threshold.
    '''
    
    proportions = df.groupby(groupby_columns).size() / df.shape[0]
    return proportions[proportions < threshold]

In [33]:
get_anomalies(df, ['day_of_week', 'hour'], 0.01)

day_of_week  hour
Tuesday      17      0.000143
             18      0.000572
Wednesday    0       0.001073
             4       0.000072
             6       0.005009
dtype: float64